In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import ipywidgets as widgets
#jupyter nbextension enable --py widgetsnbextension
from ipywidgets import interact#, interact_manual
import itables.options as opt
opt.maxBytes = 0
opt.lengthMenu = [20,100,500]
opt.columnDefs = [
    {"targets": "_all", "width": "150px", "className": "dt-wrap dt-left"},
]

#!pip3 install itables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=False)

In [ ]:
import pandas as pd
# Set the display options
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.expand_frame_repr', False)  # Do not wrap DataFrame
pd.set_option('max_colwidth', None)  # Display full column width

In [ ]:
#!pip3 install --upgrade numpy
#!pip3 install numba==0.53

In [ ]:
#!pip3 install --upgrade pandas
#!pip3 install --upgrade nltk
#!pip3 install --upgrade textblob
#!pip3 install ipython-autotime
#!pip3 install umap-learn[plot]
#!pip3 install hdbscan
import nltk
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('brown')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('vader_lexicon')
#!pip3 install --upgrade requests
#!pip3 install sentence_transformers
#!pip3 install tensorflow
#!pip3 install tensorflow_hub
#!pip3 install spaCy
#!python3 -m spacy download en_core_web_sm
#!pip3 install pandarallel
#!pip3 install imagehash
#!pip3 install simplejson

#plots
#!pip3 install matplotlib
#!pip3 install datashader
#!pip3 install bokeh
#!pip3 install holoviews
#!pip3 install pyenchant
#!pip3 install git+https://github.com/casics/nostril.git


In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
import nltk
from textblob import TextBlob
from tqdm.notebook import tqdm
tqdm.pandas()
import requests
from pandarallel import pandarallel
import umap.umap_ as umap
import hdbscan
import imagehash
import spacy
from collections import Counter
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import random
import itertools
import json

from scipy import spatial

# to plot
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LogNorm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import umap
%matplotlib inline
import random
import enchant
from numpy.linalg import norm


#topic modeling
import re
from nltk.corpus import brown
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk

## Clean non english labels function

#### Lemmatization

In [ ]:
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def lemmatize_sentence(list_of_words):
    # Create WordNetLemmatizer object
    wnl = WordNetLemmatizer()

    for i in range(len(list_of_words)):
        #print (list_of_words[i] + " ---> " + wnl.lemmatize(list_of_words[i]))
        list_of_words[i] = wnl.lemmatize(list_of_words[i])
    list_of_words.sort()
    return list_of_words

In [ ]:
my_dict = enchant.Dict("en")
it_words = ['accept', 'active', 'admin', 'apikey', 'array', 'attacks', 'austria', 'autosizes', 'bytes', 'captcha', 'change', 'chat', 'chris', 'client', 'client', 'columheaders', 'czechia', 'dateline', 'dos', 'ddos', 'debug', 
            'description', 'displayorder', 'dns', 'dport', 'drop', 'email', 'encrypt', 'event', 'facebook', 'false', 'filter', 'following', 'forumid', 'forward', 'frame', 'freebsd', 'function', 'germany', 'get', 'gmail', 
            'gnulinux', 'hash', 'height', 'hpux', 'icmp', 'icq', 'iframe', 'input', 'instead', 'ipaddress', 'iptables', 'irix', 'jabber', 'kernel', 'key', 'keyarg', 'keycode', 'keylogger', 'keys', 'let', 
            'linux', 'listening', 'loopback', 'martin', 'masquerade', 'md5', 'microsoft', 'mysql', 'nat', 'netherlands', 'options', 'output', 'pass', 'pass', 'php', 'pluto', 'policy', 'port', 'postid', 'postrouting', 
            'powershell', 'prerouting', 'pro', 'product', 'product', 'public', 'quick', 'random', 'redirect', 'registration', 'return', 'romania', 'root', 'rsa', 'script', 'server', 'skype', 'smb', 'smp', 'sms', 'src', 
            'startup', 'state', 'states', 'stratus', 'sudo', 'switzerland', 'taglist', 'tcp', 'threadid', 'timeout', 'title', 'trendmicro', 'type', 'udp', 'unicode', 'united', 'url', 'usergroupid', 
            'userid', 'username', 'var', 'varname', 'visible', 'vmware', 'windows', 'x64', 'xml', 'zynamics']

## adding the lemmatized word for every it word to it words, so that they are recognized when comparing them with LDA
lemm_it_words = it_words.copy()
lemm_it_words = lemmatize_sentence(lemm_it_words)
it_words.extend(lemm_it_words)
it_words.sort()

def split_cluster_labels_and_check_en(cluster_label, threshold): #can be used for list of cluster_labels or list of lda topics
    try:
        word_list = split_cluster_label_to_wordlist(cluster_label)
    except: # coming from LDA, input already a list
        word_list = cluster_label
    amount_of_non_en_words = 0
    for word in word_list:
        try:
            if (my_dict.check(word)) == False and word not in it_words:
                    amount_of_non_en_words = amount_of_non_en_words +1
        except: amount_of_non_en_words = amount_of_non_en_words +1
    result = (amount_of_non_en_words/len(word_list))*100 
    if (result >= threshold): #if minimum % or more are not recognized 
        print(word_list)
        return False
    return True

def split_cluster_label_to_wordlist(cluster_label):
    return cluster_label.split("_")    

def delete_non_en_clusterlabels(dataframe, threshold):
    non_en_cluster_names = []
    cluster_label_list =list(dataframe.groupby(['cluster_label']).count().reset_index().cluster_label)
    for cluster_label in cluster_label_list:
        if split_cluster_labels_and_check_en(cluster_label, threshold) == False:
            print(cluster_label)
            non_en_cluster_names.append(cluster_label)
            dataframe =dataframe[dataframe['cluster_label']!=cluster_label]
    print(f"Amount of non-en_cluster_labels sorted out {len(non_en_cluster_names)}")
    return dataframe

# ++++++++++++++++++++
# Preparation parameters
# ++++++++++++++++++++

In [ ]:
path_inputs = "/"
path_outputs = "/"
path_figures = "/"

In [ ]:
darkjargon_df = pd.read_csv(f'{path_inputs}keyword_lists/darkjargon-net_verified.csv')
darkjargon_df = darkjargon_df.drop(['Unnamed: 0'], axis = 1)
darkjargon_df = darkjargon_df.drop(['definition source'], axis = 1)
darkjargon_df["source"] = "darkjargon"
darkjargon_df = darkjargon_df.rename(columns = {'dark term':'term'})

ncsc_df = pd.read_csv(f'{path_inputs}/keyword_lists/ncsc_glossary.csv')
ncsc_df = ncsc_df.rename(columns = {'0':'term'})
ncsc_df["source"] = "ncsc"

keywords_df = pd.concat([darkjargon_df, ncsc_df],ignore_index=True, axis=0)
keywords_df

# ++++++++++++++++++++
# LOAD CLUSTERED DATA
# ++++++++++++++++++++

Cleaning:
- outlier points
- all clusters where cluster_label is 100% not english

In [ ]:
load_subforum_df = False

def clean_dataframe(dataframe):
    dataframe = dataframe[dataframe["cluster"]!=-1] #delete the -1 cluster
    dataframe = delete_non_en_clusterlabels(dataframe, threshold = 100) #delete all clusters where clusterlabel contains only non en words
    del dataframe['all_mpnet_base_v2'] #delete this column because it's not needed atm 
    del dataframe['all-mpnet-base-v2_reduced'] #delete this column because it's not needed atm
    return dataframe

In [ ]:
dataframe_all_forums_en = pd.read_csv(f'{path_inputs}dataframes/dataframe_all_forums_clustering_and_labeling_completed_en_only.csv')
dataframe_all_forums_ru = pd.read_csv(f'{path_inputs}dataframes/dataframe_all_forums_clustering_and_labeling_completed_ru_only.csv')
dataframe_all_forums_ru = clean_dataframe(dataframe_all_forums_ru)
dataframe_all_forums_en = clean_dataframe(dataframe_all_forums_en)

# Topic Modeling

- tokanizing text
- Removing stopwords
- Applying lemmatization on every line of text
- Applying LDA

## Clean lda

Cleaning all clusters where less than 20 lda keywords
And if more 80% of keywords not recognized by en dict, working with lemmatizer

In [ ]:
NUM_TOPICS = 1
np.random.seed(42)
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
additional_stopwords =["new", "get", "one", "like", "find", "get", "also", "like", "good", "tell", "help", "give", "add", "one", "let", "well", "need", "help", "found", "much", "next", "kind", "use", "find", "ask", "maybe", "make", "tell", "way", "mean", "many", "may", "ok", "okk", "best", "better", "hello", "know", "greets", "greetz", "shit", "thing", "nothing", "say" ]
lda_error_counter = 0

STOPWORDS.update(additional_stopwords)


def clean_and_lemm_text(text):
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z-][a-zA-Z-]{2,}', t)]
    cleaned_text = lemmatize_sentence(cleaned_text)
    return cleaned_text


def build_topic_models(data, amount_of_topics):
    # tokenize data and filter stopwords
    tokenized_data = []
    for text in data:
        tokenized_data.append(clean_and_lemm_text(text))
    # Build a Dictionary
    dictionary = corpora.Dictionary(tokenized_data)
    # Transform the texts to numerical form
    corpus = [dictionary.doc2bow(text) for text in tokenized_data]
    lda_topics = []
    global lda_error_counter
    try:
        lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary, random_state=np.random.RandomState(42))
        for topic in lda_model.show_topic(0, amount_of_topics):
            lda_topics.append(topic[0])
    except Exception as e:
        print(e)
        lda_topics = []
        lda_error_counter = lda_error_counter +1
    lda_topics.sort()
    
    return lda_topics
                                 

# takes dataframe as input and returns dataframe with topics
def apply_lda_to_dataframe(dataframe, amount_of_topics):
    dataframe_topics = pd.DataFrame()
    dataframe_topics['cluster'] = list(set(dataframe['cluster']))
    
    def generate_lda(cluster_nr):
        df = dataframe[dataframe["cluster"] == cluster_nr]
        data = list(df["body"])
        lda_topics = build_topic_models(data, amount_of_topics)
        return pd.Series([lda_topics])

    dataframe_topics[["lda_topics"]] = dataframe_topics.progress_apply(lambda x: generate_lda(x['cluster']), axis = 1)
    return dataframe_topics


# this function filters if a list is shorter than 20
def clean_topics(dataframe, amount_of_min_topics, en_words_threshold):
    dataframe = dataframe[dataframe.apply(lambda x: len(x['lda_topics'])>=amount_of_min_topics, axis = 1)] # clean lda topics where less than 20 keywords
    dataframe = dataframe[dataframe.apply(lambda x: split_cluster_labels_and_check_en(x["lda_topics"], en_words_threshold), axis = 1)]
    return dataframe

In [ ]:
#Generate DF & generate LDA
dataframe_topics_en = apply_lda_to_dataframe(dataframe_all_forums_en, amount_of_topics = 20)

#Set cluster index to cluster numbers (necessary for calculation of similarity matrix)
dataframe_topics_en=dataframe_topics_en.set_index("cluster")
dataframe_topics_en.index.names = ["cluster_en"]
dataframe_topics_en = clean_topics(dataframe_topics_en, amount_of_min_topics = 20, en_words_threshold = 80)

In [ ]:
#Generate DF & generate LDA
dataframe_topics_ru = apply_lda_to_dataframe(dataframe_all_forums_ru, amount_of_topics = 20)

#Set cluster index to cluster numbers (necessary for calculation of similarity matrix)
dataframe_topics_ru =dataframe_topics_ru.set_index("cluster")
dataframe_topics_ru.index.names = ["cluster_ru"]
dataframe_topics_ru = clean_topics(dataframe_topics_ru, amount_of_min_topics = 20, en_words_threshold = 80)

## Vectorize keywords to compare dataframes

Generating 1 keyword vector of all dataframes

In [ ]:
def generate_whole_keyword_vector(column, df_en, df_ru):
    set_of_all_keywords = set([])
    df_en.progress_apply(lambda x: set_of_all_keywords.update(x[column]), axis = 1)
    df_ru.progress_apply(lambda x: set_of_all_keywords.update(x[column]), axis = 1)
    return set_of_all_keywords

# Generates the keyword vector 
def generate_cluster_keyword_vector(keyword_list_of_cluster, keyword_vector):
    cluster_vector = []
    for keyword in keyword_vector:
        if keyword in keyword_list_of_cluster:
            cluster_vector.append(1)
        else:
             cluster_vector.append(0)
    return cluster_vector

In [ ]:
keyword_vector = list(generate_whole_keyword_vector("lda_topics", dataframe_topics_en, dataframe_topics_ru))
keyword_vector.sort()

For every cluster in the dataframe, generate a cluster vector.

In [ ]:
dataframe_topics_en["keyword_vector"] = dataframe_topics_en.progress_apply(lambda x: generate_cluster_keyword_vector(x['lda_topics'], keyword_vector), axis = 1)
dataframe_topics_ru["keyword_vector"] = dataframe_topics_ru.progress_apply(lambda x: generate_cluster_keyword_vector(x['lda_topics'], keyword_vector), axis = 1)

In [ ]:
dataframe_topics_en.to_csv(f'{path_outputs}dataframe_topics_en.csv')
dataframe_topics_ru.to_csv(f'{path_outputs}dataframe_topics_ru.csv')

In [ ]:
dataframe_topics_en = pd.read_csv(f'{path_outputs}dataframe_topics_en.csv')
dataframe_topics_ru = pd.read_csv(f'{path_outputs}dataframe_topics_ru.csv')

## Cosine similarity LDA

Uses the LDA keywords as vector and computes the cosine similarity to create the similarity matrix

In [ ]:
def plot_cosine_matrix_to_cdf(cosine_similarity_matrix, main_dataframe):
    sns.set_theme(font_scale=font_scale_for_figures, style = "white")
    forums_in_cluster = cosine_similarity
    n_bins = len(forums_in_cluster.keys())  #nr of clusters
    x = list(forums_in_cluster.values())
    fig, ax = plt.subplots(figsize=(16, 8))
    n, bins, patches = ax.hist(x, n_bins, density=True, histtype='step', cumulative=True, label='Cosine similarity', linewidth=linewidth)

    ax.grid(True)
    ax.legend(loc='right')
    ax.set_title(f'Cosine similarity per cluster in {main_dataframe} dataframe')
    ax.set_xlabel('Cosine similarity')
    ax.set_ylabel('Clusters')
    
    plt.savefig(f"{path_figures}cosine_similarity_for_clusters_in_{main_dataframe}_dataframe.png", bbox_inches='tight')
    plt.show()

def prepare_matrix_for_plotting(cosine_similarity_matrix):
    cosine_similarity_matrix.progress_apply(lambda x: cosine_of_two_vectors(keyword_vec_df1, x['keyword_vector']), axis = 1) #need to transfer the matrix to a dict, with keys = index, values = values at that index
    
#---

def cosine_of_two_vectors(keyword_vec_df1, keyword_vec_df2):
    cosine = np.dot(keyword_vec_df1, keyword_vec_df2)/(norm(keyword_vec_df1)*norm(keyword_vec_df2))
    return round(cosine,2)

def calc_cosine_distance(keyword_vec_df1, dataframe_2_kw_vec):   
    return dataframe_2_kw_vec.apply(lambda x: cosine_of_two_vectors(keyword_vec_df1, x))

def compare_cosine_similarity_lda(dataframe1, dataframe2):    
    cosine_similarity_matrix = pd.DataFrame()
    cosine_similarity_matrix = dataframe1["keyword_vector"].progress_apply(lambda x: calc_cosine_distance(x, dataframe2["keyword_vector"]))
    return cosine_similarity_matrix

In [ ]:
cosine_similarity_matrix = compare_cosine_similarity_lda(dataframe_topics_ru, dataframe_topics_en)

In [ ]:
cosine_similarity_matrix.to_csv(f'{path_outputs}cosine_similarity_matrix.csv')
cosine_similarity_matrix = pd.read_csv(f'{path_outputs}cosine_similarity_matrix.csv')

In [ ]:
sns.set_theme(font_scale=3, style = "white")
fig, ax = plt.subplots(figsize=(300, 460))
h = sns.heatmap (data = cosine_similarity_matrix, annot=True, annot_kws={'rotation': 0, "fontsize":30}, norm=LogNorm(), fmt='g', cmap="YlGnBu")
h.invert_yaxis()
plt.xlabel('Clusters English', fontsize = 50) # x-axis label with fontsize 15
plt.ylabel('Clusters Russian', fontsize = 50) # x-axis label with fontsize 15
plt.savefig(f"{path_figures}cosine_matrix_heatmap_LDA.png", transparent=False, bbox_inches='tight')

In [ ]:
dataframe_topics_en["mean"] =cosine_similarity_matrix.mean(axis=0)
dataframe_topics_en["max"] =cosine_similarity_matrix.max(axis=0)

dataframe_topics_ru["mean"] =cosine_similarity_matrix.mean(axis=1)
dataframe_topics_ru["max"] =cosine_similarity_matrix.max(axis=1)

In [ ]:
del dataframe_topics_en["keyword_vector"]
del dataframe_topics_ru["keyword_vector"]

# Checking POK

## POK Russian

In [ ]:
init_notebook_mode(all_interactive=True)
dataframe_topics_ru[dataframe_topics_ru["max"]<=0.15].sort_values('max')

#### Generating a list, values sorted by max, so content can be easily checked

In [ ]:
clusters_max_15_ru = list(dataframe_topics_ru[dataframe_topics_ru["max"]<=0.15].sort_values('max').index)
clusters_max_15_ru

In [ ]:
init_notebook_mode(all_interactive=False)
@interact
def show_articles_more_than(column='cluster', x=list(clusters_max_15_ru)):
    return dataframe_all_forums_ru.loc[dataframe_all_forums_ru[column] == x]

#### To check if certain topics also appear in the en dataframe

In [ ]:
dataframe_all_forums_en[dataframe_all_forums_en["body"].str.contains("example")]

## POK English

In [ ]:
init_notebook_mode(all_interactive=True)
dataframe_topics_en[dataframe_topics_en["max"]<=0.15].sort_values('max')

In [ ]:
clusters_max_15_en = list(dataframe_topics_en[dataframe_topics_en["max"]<=0.15].sort_values('max').index)

In [ ]:
@interact
def show_articles_more_than(column='cluster', x=list(clusters_max_15_en)):
    return dataframe_all_forums_en.loc[dataframe_all_forums_en[column] == x]

#### To check if certain topics also appear in the ru dataframe

In [ ]:
dataframe_all_forums_ru[dataframe_all_forums_ru["body"].str.contains("example")]

# Comparing the clusters
To evaluate the similarity pairs, we define 3 categories for the relation of the two topics discussed in the two clusters:
- very related (similarity > 0.35)
- somewhat related (similarity 0.35 - 0.2)
- not related (similarity <0.2)

In [ ]:
import math

def get_similarities_of_matrix(cosine_similarity_matrix):
    sample_d = {} 
    possible_cosine_outcome = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
    possible_cosine_outcome.reverse()
    i = 0
    for i in range (len(possible_cosine_outcome)):
        newDict = {}
        for (key, value) in cosine_similarity_matrix.stack().to_dict().items():
            if (value== possible_cosine_outcome[i]): # and (value< possible_cosine_outcome[i+1]):
                newDict[(key)] = value
        print(f"Cosine similarity {possible_cosine_outcome[i]}")
        keys = len(newDict.keys())
        print(f"Amount of findings: {keys}")

        keys = newDict.keys() 
        sample_d.update({k: newDict[k] for k in keys})

        print(f"{possible_cosine_outcome[i]} done \n")
    return sample_d

### Cosine similarities of all topics

In [ ]:
similarities = get_similarities_of_matrix(cosine_similarity_matrix)

In [ ]:
init_notebook_mode(all_interactive=False)

dataframe_similarities = pd.DataFrame.from_dict(similarities, orient = "index", columns = ["similarity"]).reset_index()
dataframe_similarities[['ru_cluster', 'en_cluster']] = dataframe_similarities['index'].progress_apply(lambda x: pd.Series(x))
dataframe_similarities

In [ ]:
pandarallel.initialize(progress_bar=True, nb_workers = 50)
dataframe_common_topics = dataframe_similarities[dataframe_similarities["similarity"]>=0.4]
dataframe_unclear_topics = dataframe_similarities[(dataframe_similarities["similarity"]<=0.35) & (dataframe_similarities["similarity"]>= 0.2)]
dataframe_distinct_topics = dataframe_similarities[dataframe_similarities["similarity"]<0.2]

def get_label(row):
    ru_label = set(dataframe_all_forums_ru[dataframe_all_forums_ru.cluster == row.ru_cluster]["cluster_label"])
    en_label = set(dataframe_all_forums_en[dataframe_all_forums_en.cluster == row.en_cluster]["cluster_label"])
    return ru_label, en_label
dataframe_common_topics[['ru_label', 'en_label']] = dataframe_common_topics.progress_apply(lambda x: pd.Series(get_label(x)), axis=1)

#dataframe_similarities

In [ ]:
dataframe_common_topics.to_csv(f'{path_outputs}dataframe_common_topics.csv')
dataframe_unclear_topics.to_csv(f'{path_outputs}dataframe_unclear_topics.csv')
dataframe_distinct_topics.to_csv(f'{path_outputs}dataframe_distinct_topics.csv')

In [ ]:
dataframe_common_topics